In [1]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from ftplib import FTP_TLS
import tkinter as tk
from tkinter import messagebox

delta=dt.timedelta(days=1)

In [2]:
# Función para asignar los días de la semana a cada fecha, si es festivo se trata como un domingo
import holidays
co_holidays = holidays.Colombia()

def typedays(row,tipo):

     if tipo=='WeekDay':
          return row['Fecha'].weekday()
     
     elif tipo=='WeekMonth':
          return (row['Fecha'].day - 1) // 7 + 1
     
     elif tipo=='DayType':
          if row['Fecha'] in co_holidays:
               return 3
          elif row['Fecha'].weekday()==5:
               return 2
          elif row['Fecha'].weekday()==6:
               return 3
          else:
               return 1

In [3]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaDesp=r'C:\Información XM\\Publico\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [4]:
# Rango de análisis

# s_FechaIni='2022-11-19'
# s_FechaFin='2024-11-18'

s_FechaIni='2024-12-21'
s_FechaFin='2025-01-17'

tipod='day_w'

#Camniar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

# Create a DataFrame of dates
df_dates = pd.DataFrame(pd.date_range(start=s_FechaIni, end=s_FechaFin, freq='D')  , columns=["Date"])

df_dates['Mes']=df_dates['Date'].dt.month

df_dates

,Date,Mes
0,2024-12-21,12
1,2024-12-22,12
2,2024-12-23,12
3,2024-12-24,12
4,2024-12-25,12
5,2024-12-26,12
6,2024-12-27,12
7,2024-12-28,12
8,2024-12-29,12
9,2024-12-30,12


In [5]:
# Función para descargar el archivo
def DownFile(fecha_dt,UsuXM,PwsXM,Tipo):

    # Connect to the FTP server (replace with your actual details)
    ftps  = FTP_TLS()
    ftps .connect('xmftps.xm.com.co', 210)  # default port is 210

    # Secure the control connection
    ftps .auth()
    ftps .prot_p()  # Switch to secure data connection (important!)

    ftps .login(UsuXM, PwsXM)

    # Obtener mes y día de la fecha inicial
    # Transformar string en fecha
    # fecha = dt.datetime.strptime(fecha_dt, "%d/%m/%Y")

    # Obtener mes y día
    year= fecha_dt.year
    mes = fecha_dt.month
    dia = fecha_dt.day

    if Tipo=='DemDespacho':
        # Navigate to the directory you want to access
        ftps.cwd(rf"/INFORMACION_XM/Publico/DESPACHO/{year:04d}-{mes:02d}")
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day
        ftps.cwd(rf"/INFORMACION_XM/PUBLICO/DEMANDAS/Pronostico Oficial/{year:04d}-{mes:02d}")
    else:
        messagebox.showinfo('Estado del proceso',f'No se reconoce el formato {Tipo}', parent=root)
        df=pd.DataFrame()
        return df


    # List files
    files = ftps.nlst()
    # print("Available files:", files)


    if Tipo=='DemDespacho':
        # Download condiciones iniciales de planta
        pathfile=rf"C:\Información XM\Publico\DESPACHO\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"dDEM{mes:02d}{dia:02d}.TXT"    
    elif Tipo=='Demanda':
        # Calcular el lunes asociado a la semana de la fecha dada
        lunes_asociado = fecha_dt - dt.timedelta(days=fecha_dt.weekday())
        # Obtener mes y día
        year= lunes_asociado.year
        mes = lunes_asociado.month
        dia = lunes_asociado.day

        pathfile=rf"C:\Información XM\PUBLICO\DEMANDAS\Pronostico Oficial\{year:04d}-{mes:02d}"
        if not os.path.exists(pathfile):
            os.makedirs(pathfile)
        filename = rf"PRON_AREAS{mes:02d}{dia:02d}.TXT"

    try:
        # print(pathfile + "\\" + filename)
        with open(pathfile + "\\" + filename, 'wb') as f:
            ftps.retrbinary(f"RETR {filename}", f.write)

        ftps.quit()
        # print(f"{filename} downloaded successfully.")


    except:

        df=pd.DataFrame()

    
    return 

In [22]:
folderpath=Path(sRutaDesp + 'Despacho\\')

filetype='dDEM'

l_colDem=['Subarea']

l_SubCar=['ATLANTIC','BOLIVAR','GCM','CORDOSUC','CERROMAT'] # Lista con las subareas Caribe

# df_Factor=pd.DataFrame()
# df_Factor_typo_d=pd.DataFrame()

for i in range(1,25): 
    l_colDem.append(i)

df_Dem=pd.DataFrame()
df_datesAx=df_dates.copy()
df_datesAx=df_datesAx.sort_values(by='Date', ascending=True)

# print(mes)

for ind in df_datesAx.index:

    dFecha=df_datesAx.at[ind,'Date']

    ano=dFecha.year
    mes=dFecha.month
    dia=dFecha.day

    UsuXM='1060588666'
    PwsXM='Alejo230708*+'
    Tipo='DemDespacho'
    DownFile(dFecha,UsuXM,PwsXM,Tipo)

    
    file = filetype + "{:02d}".format(mes) + "{:02d}".format(dia) + '.txt'
    s_filepath=folderpath.joinpath(str(ano) + '-' + "{:02d}".format(mes),file)
    df_Aux = pd.read_csv(s_filepath, sep=',',names=l_colDem,encoding="ISO-8859-1")
    df_Aux['Fecha']=dFecha
    df_Aux = df_Aux[~df_Aux['Subarea'].isin(['Total','ECUADOR138','ECUADOR220','COROZO','CUATRIC'])]


    df_Dem=pd.concat([df_Dem,df_Aux])


# df_Dem.to_csv(sRutaPrint.joinpath('D1.csv'))

df_DemAux=df_Dem.copy()

In [26]:
df_Dem=df_DemAux.copy()

# Normalizar y reemplazar subáreas que no están en l_SubCar por "INTERIOR"
df_Dem['Subarea'] = df_Dem['Subarea'].astype(str).str.strip()
caribe_up = [s.upper() for s in l_SubCar]
mask = df_Dem['Subarea'].str.upper().isin(caribe_up)
df_Dem.loc[~mask, 'Subarea'] = 'INTERIOR'

# Definir tipo de día
df_Dem['day_osf']=df_Dem.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
# Definir día de la semana
df_Dem['day_w']=df_Dem.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)
# Definir semana del mes
# df_Dem['week_m']=df_Dem.apply(lambda row: typedays(row,tipo='WeekMonth'),axis=1)

# Definir los periodos en una columna
df_Dem = pd.melt(df_Dem, id_vars=['Fecha','day_osf','day_w','Subarea'], var_name='Periodo', value_name='DemMW')


df_Dem['Mes']=df_Dem['Fecha'].dt.month
df_Dem['Dia']=df_Dem['Fecha'].dt.day

# Cálculo de la demanda por subárea
df_Dem=df_Dem.groupby(['Fecha','Mes','day_osf','day_w','Subarea','Periodo'])[['DemMW']].sum()
df_Dem=df_Dem.reset_index()
df_Dem['DemMW']=df_Dem['DemMW'].round(2)





# df_Factor_typo_d.to_csv(sRutaPrint.joinpath('Factores.csv'))


In [25]:
df_Dem

,Fecha,Mes,day_osf,day_w,Subarea,Periodo,DemMW
0,2024-12-21,12,2,5,ATLANTIC,1,734.24
1,2024-12-21,12,2,5,ATLANTIC,2,713.01
2,2024-12-21,12,2,5,ATLANTIC,3,689.94
3,2024-12-21,12,2,5,ATLANTIC,4,670.26
4,2024-12-21,12,2,5,ATLANTIC,5,657.42
...,...,...,...,...,...,...,...
4027,2025-01-17,1,1,4,INTERIOR,20,8051.87
4028,2025-01-17,1,1,4,INTERIOR,21,7849.11
4029,2025-01-17,1,1,4,INTERIOR,22,7129.73
4030,2025-01-17,1,1,4,INTERIOR,23,6581.07


In [13]:
mask

0     False
1     False
2      True
3     False
4      True
5     False
6     False
7     False
8      True
9      True
11    False
15     True
16    False
17    False
18    False
19    False
20    False
21    False
Name: Subarea, dtype: bool

In [15]:
df_FacDay.to_csv(sRutaPrint.joinpath('Factores.csv'),index=False)
df_FacDay

,Mes,day_w,F_Interior,F_Caribe2
0,1,0,0.709352,0.210486
1,1,1,0.718120,0.204856
2,1,2,0.719055,0.203989
3,1,3,0.718279,0.204551
4,1,4,0.718736,0.204755
...,...,...,...,...
79,12,2,0.723627,0.200986
80,12,3,0.719731,0.203825
81,12,4,0.721895,0.203563
82,12,5,0.716423,0.206292


In [6]:
# Lectura de datos de enrtrada para el cálculo
s_parentpath=Path(r'C:\Alejo\Eje de Planeación\Análisis Energético\Proyección2027-2031')
filepath=s_parentpath.joinpath(s_parentpath,'DatosProyección2027_2031.xlsx')

# Carga del nivel probabilístico del embalse
sheet_name='DemandaUPME_2044'
df_DemMes=pd.read_excel(filepath, header=0,sheet_name=sheet_name)
# df_DemMes=df_DemMes.set_index('Embalse_Sinergox')

In [7]:
df_DemMes

,Fecha,Esc. Medio,IC Superior 95%,IC Inferior 95%,IC Superior 68%,IC Inferior 68%
0,2025-04-01,6907.489142,7067.714032,6747.791688,6995.252774,6819.995742
1,2025-05-01,7160.999343,7326.904954,6995.639869,7251.874605,7070.403893
2,2025-06-01,6940.408197,7101.052420,6780.292791,7028.401519,6852.685813
3,2025-07-01,7166.581411,7402.879065,6931.385049,7296.025088,7037.701972
4,2025-08-01,7191.521672,7427.948539,6956.196699,7321.036132,7062.571759
...,...,...,...,...,...,...
172,2039-08-01,10127.926066,11920.643984,8380.940175,11111.015432,9168.251767
173,2039-09-01,9986.900923,11750.590091,8268.203263,10954.071524,9042.766239
174,2039-10-01,10211.765041,12023.523267,8446.283198,11204.753642,9242.469942
175,2039-11-01,10098.690480,11891.285889,8351.881994,11081.176322,9139.647536


In [15]:
from datetime import datetime
import pandas as pd

tipo='MP'
d_YearIni=2026
d_YearFin=2026


df_DemMod=pd.DataFrame()

for year in range(d_YearIni,d_YearFin+1):

    for mes in range(1,3):
        
        fecha_str = f"{year}-{mes:02d}-01"

        if tipo=='MP':
            fechas = pd.date_range(start=fecha_str, periods=pd.Period(fecha_str).days_in_month, freq='D')
        elif tipo=='LP':
            # Crear fechas para los primeros 5 días del mes
            fechas = pd.date_range(start=fecha_str, periods=5, freq='D')
            # Crear columna 'DiaTipo' según la posición del día en el mes
            dia_tipo_map = {0: 1, 1: 2, 2: 3, 3: 1, 4: 1}
            df_diatipo = pd.DataFrame({'Fecha': fechas, 'day_osf': [dia_tipo_map[i] for i in range(len(fechas))]})

        # Crear dataframe con 24 periodos para cada día

        df_mes = pd.DataFrame([
            {'Fecha': fecha, 'Periodo': periodo}
            for fecha in fechas
            for periodo in range(1, 25)
        ])
        
        if tipo=='MP':
            if tipod=='day_osf':
                df_mes['day_osf']=df_mes.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
            else:
                df_mes['day_w']=df_mes.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)
        elif tipo=='LP':
            df_mes = df_mes.merge(df_diatipo, on='Fecha', how='left')
        
        d_date = datetime.strptime(fecha_str, '%Y-%m-%d')
        df_DemDia=df_DemMes[(df_DemMes['Fecha']==d_date)]['IC Superior 95%']
        valor = df_DemDia.values[0]
        df_mes['ValorMes']=valor
        
        df_mes['Mes'] = df_mes['Fecha'].dt.month

        if tipod=='day_osf':
            df_mes=df_mes.merge(df_Factor_typo_d,left_on=['Mes','day_osf','Periodo'],
                                right_on=['Mes','day_osf','Periodo'], how='left')[['Fecha','Mes','day_osf','Periodo','ValorMes','FactorDay','FactorHour','F_Car','F_Interior','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']]
        else:
            df_mes=df_mes.merge(df_Factor_typo_d,left_on=['Mes','day_w','Periodo'],
                                right_on=['Mes','day_w','Periodo'], how='left')[['Fecha','Mes','day_w','Periodo','ValorMes','FactorDay','FactorHour','F_Car','F_Interior','F_ATLANTIC', 'F_BOLIVAR','F_CERROMAT', 'F_CORDOSUC', 'F_GCM']]
        
        df_mes['SubAntioquia'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_Interior']
        df_mes['SubAtlantico'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_ATLANTIC']
        df_mes['SubBolivar'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_BOLIVAR']
        df_mes['SubGCM'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_GCM']
        df_mes['SubCordoba-Sucre'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_CORDOSUC']
        df_mes['SubCerromatoso'] = df_mes['ValorMes'] * 1000 * df_mes['FactorDay'] * df_mes['FactorHour'] * df_mes['F_CERROMAT']


        df_mes=df_mes[['Fecha','Periodo','SubAntioquia','SubAtlantico','SubBolivar','SubGCM','SubCordoba-Sucre','SubCerromatoso']]

        df_mes = df_mes.melt(id_vars=['Fecha', 'Periodo'], var_name='Subarea', value_name='Valor')
        df_mes['Fecha'] = df_mes['Fecha'].dt.date

        df_DemMod=pd.concat([df_DemMod,df_mes],axis=0)


df_DemMod.to_csv(sRutaPrint.joinpath('DatosDemanda.csv'))

In [11]:

df_mes

,Fecha,Periodo,Subarea,Valor
0,2025-12-01,1,SubAntioquia,5851.511599
1,2025-12-01,2,SubAntioquia,5647.864868
2,2025-12-01,3,SubAntioquia,5537.822059
3,2025-12-01,4,SubAntioquia,5529.560141
4,2025-12-01,5,SubAntioquia,5723.854056
...,...,...,...,...
4459,2025-12-31,20,SubCerromatoso,263.608080
4460,2025-12-31,21,SubCerromatoso,263.400856
4461,2025-12-31,22,SubCerromatoso,256.772673
4462,2025-12-31,23,SubCerromatoso,256.611386


In [11]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 

s_path=Path.cwd()
s_prt_path=s_path.parent

objetoAPI = pydataxm.ReadDB()                    #Se almacena el servicio en el nombre objetoAPI
df =objetoAPI.get_collections()   

FechaIni=dt.date(2024, 9, 1)
FechaFin=dt.date(2024, 11, 30)

In [ ]:
# Unit kWh
df_Dem= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaSIN",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

### Cálcular día

In [ ]:
# Rango de análisis

s_FechaIni='2024-11-25'
s_FechaFin='2024-12-1'

#Cambiar a formato fecha
d_FechaIni=dt.datetime.strptime(s_FechaIni,'%Y-%m-%d')
d_FechaFin=dt.datetime.strptime(s_FechaFin,'%Y-%m-%d')

# Create a DataFrame of dates
df_dates = pd.DataFrame(pd.date_range(start=s_FechaIni, end=s_FechaFin, freq='D')  , columns=["Fecha"])

df_dates['Mes']=df_dates['Fecha'].dt.month

# Definir tipo de día
df_dates['day_osf']=df_dates.apply(lambda row: typedays(row,tipo='DayType'),axis=1)
# Definir día de la semana
df_dates['day_w']=df_dates.apply(lambda row: typedays(row,tipo='WeekDay'),axis=1)

In [157]:
df_dates

,Fecha,Mes,day_osf,day_w
0,2024-11-25,11,1,0
1,2024-11-26,11,1,1
2,2024-11-27,11,1,2
3,2024-11-28,11,1,3
4,2024-11-29,11,1,4
5,2024-11-30,11,2,5
6,2024-12-01,12,3,6


In [ ]:
# Definir el valor de energía del mes de la UMPE
ValorMes=10000

periods = [i for i in range(1,25)]  # Create 24 period labels

df_DemRes=pd.DataFrame()

for ind in df_dates.index:
    date=df_dates.loc[ind,'Fecha']

    Mes=date.month
    day_osf=df_dates.loc[ind,'day_osf']
    day_w=df_dates.loc[ind,'day_w']


    # Validar si esciste el día de la semana para el festivo que se está presentando, sino acercar al festivo que se tenga
    # Si es semana validar si hay otro en semana, sino tomar el domingo
    if day_osf==3:
        a=1
    

    df_Fac=df_Factor[(df_Factor.Mes==Mes) & (df_Factor.day_osf==day_osf) & (df_Factor.day_w==day_w)]

    df_Axu=pd.DataFrame({'Periodo': periods})
    df_Axu['Fecha']=date


df_Axu
    # print(date)

,Periodo,Fecha
0,1,2024-12-01
1,2,2024-12-01
2,3,2024-12-01
3,4,2024-12-01
4,5,2024-12-01
5,6,2024-12-01
6,7,2024-12-01
7,8,2024-12-01
8,9,2024-12-01
9,10,2024-12-01


### Factores con demanda real

In [1]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 
import datetime as dt                            
import pandas as pd
from pathlib import Path
import os

In [2]:
s_path=Path.cwd()
s_prt_path=s_path.parent
s_pathPrint=s_path.joinpath('Print')

objetoAPI = pydataxm.ReadDB()                    #Se almacena el servicio en el nombre objetoAPI
df =objetoAPI.get_collections()  

FechaIni=dt.date(2023, 10, 1)
FechaFin=dt.date(2025, 9, 30)

In [3]:
df_DemRealIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaReal",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta

df_DemSINIni= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "DemaSIN",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
df_DemSINIni.rename(columns={'Value':'DemSIN'},inplace=True)
df_DemSINIni['DemSIN']=round(df_DemSINIni['DemSIN']/1e6,2)

In [4]:
df_DemReal=df_DemRealIni.copy()

l_per=[]
for i in range(1,25):
    if i<10:
        df_DemReal.rename(columns={'Values_Hour0' + str(i): i}, inplace=True)
        l_per.append(i)
    else:
        df_DemReal.rename(columns={'Values_Hour' + str(i): i}, inplace=True)
        l_per.append(i)

# df_DemReal=df_DemReal.groupby(['Date'])[l_per].mean()
# Get mean of numeric columns
df_DemReal_mean = pd.DataFrame(df_DemReal[l_per].mean())
Total=df_DemReal_mean[[0]].sum()
df_DemReal_mean['Factor'] = df_DemReal_mean[0] / Total[0]
df_DemReal_mean.to_csv(s_pathPrint.joinpath('Factores.csv'))

# df_DemReal['TotalDem']=0
# for col in l_per:
#     df_DemReal['TotalDem']=df_DemReal['TotalDem']+df_DemReal[col]

In [5]:
# Definir los periodos en una columna
# df_DemSIN = pd.melt(df_DemSIN, id_vars=['Fecha'], var_name='Periodo', value_name='DemMW')
# # df_dataP['Precio']=df_dataP['Precio']/1000
df_DemSIN=df_DemSINIni.copy()

df_DemSIN['Yaer']=df_DemSIN['Date'].dt.year
df_DemSIN['Mes']=df_DemSIN['Date'].dt.month
df_DemSIN['Dia']=df_DemSIN['Date'].dt.day

df_DemSIN['Year-Mes']=df_DemSIN['Yaer'].astype(str) + '-' + df_DemSIN['Mes'].astype(str)

df_DemSIN=df_DemSIN.groupby(['Year-Mes'])[['DemSIN']].sum()


In [6]:
df_DemSIN.to_csv('DemandaSIN.csv')